# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifyks
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifyks')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Define the input CSV file
file = 'event_datafile_new.csv'

In [12]:
df_events_data = pd.read_csv(file)

In [13]:
df_events_data.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

## <font color='blue'>Query 1</font>
### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Step 1 - Create the table 'songs_and_sessions'

- In our table ***songs_and_sessions***, we use a compound primary key with the column **sessionId** as the partition key, and **itemInSession** as the clustering column.
- Based on the query to be formulated, the other columns in our table are **artistName**, **songName** and **songLength**.

In [9]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
session.execute("""
    CREATE TABLE IF NOT EXISTS songs_and_sessions (
        sessionId int,
        itemInSession int,
        artistName text,
        songName text,
        songLength float,
        PRIMARY KEY(sessionId, itemInSession))
    """)                    

#### Step 2 - Insert the records (relevant columns) into the table 'songs_and_sessions'

In [10]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_and_sessions (sessionId, itemInSession, artistName, songName, songLength)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Step 3 - Do a SELECT to verify that the data have been inserted into each table

To validate our use case based on the above query, we would use the below CQL query.<br>
*SELECT artistName, songName, songLength FROM songs_and_sessions WHERE sessionId = 338 AND itemInSession = 4*

In [11]:
# Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("""
    SELECT artistName, songName, songLength FROM songs_and_sessions WHERE sessionId = 338 AND itemInSession = 4 
    """)

for row in rows:
    print(row.artistname, row.songname, row.songlength)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Step 4 - Validating the results

We can validate the result of our CQL query by comparing it with the data in our pandas dataframe object (loaded with the data in our CSV file).

In [16]:
# Let us validate the result from our pandas dataframe loaded with the events data
df_results = df_events_data[(df_events_data['sessionId']==338) & (df_events_data['itemInSession']==4)]
df_results = df_results[['artist', 'song', 'length']]
df_results

,artist,song,length
444,Faithless,Music Matters (Mark Knight Dub),495.3073


## <font color='blue'>Query 2</font>
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Step 1 - Create the table 'users_and_songs'

- In our table ***users_and_songs***, we use a complex primary key, with a composite partition key. The columns **userId** and **sessionId** together form the partition key, and **itemInSession** as the clustering column.
- Based on the query to be formulated, the other columns in our table are **artistName**, **songName**, **firstName** and **lastName**.

In [17]:
## TO-DO: Query 2
session.execute("""
    CREATE TABLE IF NOT EXISTS users_and_songs (
        userId int,
        sessionId int,
        artistName text,
        songName text,
        firstName text,
        lastName text,
        itemInSession int,
        PRIMARY KEY((userId, sessionId), itemInSession))
    """)                    

#### Step 2 - Insert the records (relevant columns) into the table 'users_and_songs'

In [18]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_and_songs (userId, sessionId, artistName, songName, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

#### Step 3 - Do a SELECT to verify that the data have been inserted into each table

To validate our use case based on the above query, we would use the below CQL query.<br>
*SELECT itemInSession, artistName, songName, firstName, lastName FROM users_and_songs WHERE userId = 10 AND sessionId = 182*

In [19]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("""
    SELECT itemInSession, artistName, songName, firstName, lastName FROM users_and_songs 
    WHERE userId = 10 AND sessionId = 182 
    """)

for row in rows:
    print(row.artistname, row.songname, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Step 4 - Validating the results

We can validate the result of our CQL query by comparing it with the data in our pandas dataframe object (loaded with the data in our CSV file).

In [20]:
# Let us validate the result from our pandas dataframe loaded with the events data
df_results = df_events_data[(df_events_data['userId']==10) & (df_events_data['sessionId']==182)]
df_results = df_results[['artist', 'song', 'firstName', 'lastName']]
df_results

,artist,song,firstName,lastName
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
4705,Three Drives,Greece 2000,Sylvie,Cruz
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## <font color='blue'>Query 3</font>
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Step 1 - Create the table 'music_app_history'

- In our table ***music_app_history***, we use a primary key with the column **songName** as the partition key, and **userId** as the clustering column.
- Based on the query to be formulated, the other columns in our table are **songName**, **firstName** and **lastName**.

In [21]:
## TO-DO: Query 3
session.execute("""
    CREATE TABLE IF NOT EXISTS music_app_history (
        songName text,
        firstName text,
        lastName text,
        userId int,
        PRIMARY KEY(songName, userId))
    """)  

#### Step 2 - Insert the records (relevant columns) into the table 'music_app_history'

In [22]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (songName, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

#### Step 3 - Do a SELECT to verify that the data have been inserted into each table

To validate our use case based on the above query, we would use the below CQL query.<br>
*SELECT firstName, lastName FROM music_app_history WHERE songName = 'All Hands Against His Own'*

In [23]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
rows = session.execute("""
    SELECT firstName, lastName FROM music_app_history WHERE songName = 'All Hands Against His Own' 
    """)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### Step 4 - Validating the results

We can validate the result of our CQL query by comparing it with the data in our pandas dataframe object (loaded with the data in our CSV file).

In [25]:
# Let us validate the result from our pandas dataframe loaded with the events data
df_results = df_events_data[df_events_data['song']=="All Hands Against His Own"]
df_results = df_results[['firstName', 'lastName']]
df_results

,firstName,lastName
2792,Tegan,Levine
5135,Sara,Johnson
6298,Jacqueline,Lynch


As we can see, we were able to execute all the three queries successfully, and the query results are fully aligned with the the results from the pandas dataframe, when we apply the same conditions on it.

### Drop the tables before closing out the sessions

In [26]:
## Drop the table before closing out the sessions
try:
    session.execute("""DROP TABLE IF EXISTS songs_and_sessions""")
    session.execute("""DROP TABLE IF EXISTS users_and_songs""")
    session.execute("""DROP TABLE IF EXISTS music_app_history""")
except Exception as e:
    print("Error while dropping tables:")
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()